In [15]:
import pandas as pd
import os
import exifread as efr
from datetime import datetime

# https://gist.github.com/snakeye/fdc372dbf11370fe29eb


def _convert_to_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value:
    :type value: exifread.utils.Ratio
    :rtype: float
    """
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)


def getGPS(filepath):
    '''
    returns gps data if present other wise returns empty dictionary
    '''
    with open(filepath, 'rb') as f:
        tags = efr.process_file(f)
        latitude = tags.get('GPS GPSLatitude')
        latitude_ref = tags.get('GPS GPSLatitudeRef')
        longitude = tags.get('GPS GPSLongitude')
        longitude_ref = tags.get('GPS GPSLongitudeRef')
        if latitude:
            lat_value = _convert_to_degress(latitude)
            if latitude_ref.values != 'N':
                lat_value = -lat_value
        else:
            return {}
        if longitude:
            lon_value = _convert_to_degress(longitude)
            if longitude_ref.values != 'E':
                lon_value = -lon_value
        else:
            return {}
        return {'latitude': lat_value, 'longitude': lon_value}
    return {}

In [16]:
# set directory
directory = "photos_original"
photos = []
photos_names = []
lat_gps = []
lon_gps = []

In [17]:
# iterate over files in that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        with open(f, "rb") as photo:
            tags = efr.process_file(photo)
            photos.append(tags)
            gps = getGPS(f)
            try:
                photos_names.append(f.title())
            except:
                print("title error")
                photos_names.append("NaN")
            try:
                lat_gps.append(gps['latitude'])
            except:
                print("latitude error")
                lat_gps.append("NaN")
            try:
                lon_gps.append(gps['longitude'])
            except:
                print("longitude error")
                lon_gps.append("NaN")

File format not recognized.
File format not recognized.


Possibly corrupted field Tag 0x001A in MakerNote IFD
Possibly corrupted field Tag 0x001A in MakerNote IFD
Possibly corrupted field Tag 0x001A in MakerNote IFD
Possibly corrupted field Tag 0x001A in MakerNote IFD
Possibly corrupted field Tag 0x001A in MakerNote IFD
Possibly corrupted field Tag 0x001A in MakerNote IFD


latitude error
longitude error
latitude error
longitude error
latitude error
longitude error
latitude error
longitude error
latitude error
longitude error
latitude error
longitude error


In [18]:
photos_df = pd.DataFrame(photos)

photos_df['title'] = photos_names
photos_df['latitude'] = lat_gps
photos_df['longitude'] = lon_gps

photos_datetime = []
photos_date = []
photos_time = []
photos_combo = []
photos_name = []

# 2019:03:14 13:54:11
for index, row in photos_df.iterrows():
    dt_str = str(row['EXIF DateTimeOriginal']).replace(":", "")
    photo_name = str(row['title']).replace("Photos_Original/", "")
    photos_name.append(photo_name.lower())
    d_split = dt_str.split()
    photos_datetime.append(d_split)

for i in photos_datetime:
    try:
        photos_date.append(i[0])
    except:
        photos_date.append("NaN")
    try:
        photos_time.append(i[1])
    except:
        photos_time.append("NaN")
    try:
        photos_combo.append(i[0] + ' ' + i[1])
    except:
        photos_combo.append("NaN")

photos_df['datetime'] = pd.to_datetime(photos_combo, format="%Y%m%d %H%M%S")
photos_df['name'] = photos_name

In [19]:
photos_df.columns

Index(['Image Make', 'Image Model', 'Image Orientation', 'Image XResolution',
       'Image YResolution', 'Image ResolutionUnit', 'Image Software',
       'Image DateTime', 'Image TileWidth', 'Image TileLength',
       ...
       'MakerNote Tag 0x0028', 'MakerNote Tag 0x0012', 'EXIF CustomRendered',
       'EXIF DigitalZoomRatio', 'MakerNote HDRImageType', 'title', 'latitude',
       'longitude', 'datetime', 'name'],
      dtype='object', length=107)

In [20]:
photos_important_info_cols = ['name', 'datetime', 'latitude', 'longitude']
photos_important_info_df = pd.DataFrame(
    photos_df, columns=photos_important_info_cols)

In [21]:
photos_important_info_df.to_csv("photos_info.csv")
photos_important_info_df.to_excel(
    excel_writer="photos_info.xlsx", sheet_name="PhotosMetadata")